In [5]:
from selenium import webdriver as wd

# 아래 3개의 모듈을 한 묶음으로 많이 사용
# (기대조건이 될 때까지 기다리겠다는 의미로 활용)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지기동 전 대기시간
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

import xlsxwriter as xw
import urllib.request as req

chrome_options = Options()


# Headless 모드 : 브라우저를 안 띄우고 사용하는 모드
chrome_options.add_argument('--headless')

# webdriver 설정(Chrome, Firefox...) - Headless 모드로 설정됨
browser = wd.Chrome('./webdriver/chromedriver.exe', options=chrome_options)

# 엑셀 처리하기 위한 workbook 생성
workbook = xw.Workbook('./crawl_result.xlsx')

# 워크시트 생성
worksheet = workbook.add_worksheet()

# 일반모드(브라우저 띄움)
# browser = wd.Chrome('./webdriver/chromedriver.exe')

# 브라우저 내부 대기시간
# 랜더링하는 5초안에 브라우저가 안 띄워지면 에러발생
browser.implicitly_wait(5)

# 브라우저 사이즈
browser.set_window_size(1280, 720)

# 페이지 이동
browser.get('http://prod.danawa.com/list/?cate=112758&15main_11_02')

# 페이지 내용 확인하기
# print(f'page content : {browser.page_source}')

# 로딩이 될 때 까지 기다린 후 선택
# 더보기 클릭하기
WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH, \
                '//*[@id="dlMaker_simple"]/dd/div[2]/button[1]'))).click()

# 원하는 상품 클릭하기
WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, \
                '//*[@id="searchMaker1452"]'))).click()

time.sleep(3)

# 현재 페이지
cur_page = 1

# 크롤링 할 페이지 수 )
target_page = 3

# 엑셀의 행번호
row_num = 1

while cur_page <= target_page:
    
    # bs4 초기화
    bs = BeautifulSoup(browser.page_source, 'lxml')

    # 소스코드 정리해서 보기
    # print(bs.prettify)
    
    # 페이지 번호 출력
    print(f'******** Page : {cur_page} *******')
    print()

    prod_list = bs.select('div.main_prodlist.main_prodlist_list > ul > li')

    # print(prod_list)

    # 원하는 정보 추출
    for li in prod_list:
        if not li.find('div', class_='ad_caster'):
            # 상품명
            prod_name = li.select('p.prod_name > a')[0].text.strip()
            # 가격
            prod_price = li.select('p.price_sect > a')[0].text.strip()
            
            # 엑셀 저장
            worksheet.write(f'A{row_num}', prod_name)
            worksheet.write(f'B{row_num}', prod_price)
            
#             img_data = li.select('a.thumb_link > img')[0].get('data-original')
#             img_src = li.select('a.thumb_link > img')[0]['src']
#             print(img_data if img_data else img_src)
            row_num += 1
        print()
    print()
    
    cur_page += 1
    
    if cur_page > target_page:
        print('크롤링 성공')
        break
    
    # 페이지 클릭
    WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR , \
                f'div.number_wrap > a:nth-child({cur_page})'))).click()
    
    # 2초간 대기
    time.sleep(2)

browser.quit()
workbook.close()

******** Page : 1 *******


































******** Page : 2 *******
































******** Page : 3 *******
































크롤링 성공
